In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [46]:
# TODO 

# 0. sub_probs_old = get_p0s_p1s_p2s_p3s(window, genome, model, log=True) # utilities/get_p0s_p1s_p2s_p3s.py
# 1. alt_counts_new = np.random.multinomial(3mer_count_new, sub_probs_old, size=1)
# 2. kmer_counts_new = f(alt_counts_new)
# 3. sub_probs_new = estimate_kmer_probabilities(kmer_counts_new, args) # train/germline-model/estimate-probabilities
# 4. repeat for all 3mers, and dump dictionary to model-3-new.json 

# TODO: has reverse complementarity been handled correctly?

In [45]:
CONSTRAINT_TOOLS = '/scratch/ucgd/lustre-work/quinlan/u6018199/constraint-tools'

import pandas as pd 

pd.set_option('display.max_rows', 30)

import sys
sys.path.append(f'{CONSTRAINT_TOOLS}/utilities')

import matplotlib.pyplot as plt 
plt.rcParams.update({'font.size': 20})

import seaborn as sns 
import numpy as np 

from read_model import read_model 

def read_model_wrapper(kmer_size, train_set_label='noncoding'):
    filename = f'{CONSTRAINT_TOOLS}/dist/model-germline-grch38-Nonly.kmerSize-{kmer_size}.trainSet-{train_set_label}.json'
    model = read_model(filename)
    return model

def f(three_mer='AAA'):
    model_7 = read_model_wrapper(kmer_size=7)
    seven_mers = [seven_mer for seven_mer in model_7['kmerCounts'].keys() if seven_mer[2:5] == three_mer]
    seven_mer_counts = [model_7['kmerCounts'][seven_mer]['count'] for seven_mer in seven_mers]
    three_mer_count_new = int(np.mean(seven_mer_counts))

    # update 3mer counts AND 3mer probs
    model_3 = read_model_wrapper(kmer_size=3)
    three_mer_probabilities = model_3['kmerProbabilities'][three_mer]
    return three_mer_probabilities
    for ALT_state in three_mer_probabilities.keys(): 
        three_mer_probabilities[ALT_state] = None # TODO 
    return three_mer_probabilities
        
    np.random.multinomial(three_mer_count_new, None, size=1)
        
    three_mer_probability = None

def g(): 
    model_3 = read_model_wrapper(kmer_size=3)
    three_mers = model_3['kmerCounts'].keys()
    for three_mer in three_mers: 
        return f(three_mer)

g()

{'{C}': 0.03064242457603776,
 '{G}': 0.05273588266099003,
 '{T}': 0.017067843522482667,
 '{C,G}': 0.002132047830285883,
 '{C,T}': 0.0008130795794212962,
 '{G,T}': 0.001257811676431882,
 '{C,G,T}': 8.906225027289328e-05}